In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import Core.Nanoparticle as NP
import Core.GlobalFeatureClassifier as GFC

from LocalOpt.LocalOptimization import local_optimization
import Core.EnergyCalculator as EC

from Core.LocalEnvironmentFeatureClassifier import TopologicalEnvironmentClassifier
from Core.LocalEnvironmentCalculator import LocalEnvironmentCalculator

from MCMC import MonteCarlo

from ase.visualize import view
import pickle

In [3]:
def create_octahedron_training_set(n_particles, height, trunc, stoichiometry):
    emt_calculator = EC.EMTCalculator(fmax=0.03, steps=1000)
    top = GFC.TopologicalFeatureClassifier(list(stoichiometry.keys()))
    training_set = []
    for i in range(n_particles):
        p = NP.Nanoparticle()
        p.truncated_octahedron(height, trunc, stoichiometry)
        emt_calculator.compute_energy(p)
        training_set.append(p)
        top.compute_feature_vector(p)
        
        
    return training_set

In [4]:
def create_start_particle(height, trunc, stoichiometry):
    start_particle = NP.Nanoparticle()
    start_particle.truncated_octahedron(height, trunc, stoichiometry)
    return start_particle

In [20]:
"""Create the training set with 30 particles"""
stoichiometry={'Pt' : 56, 'Au' : 84}

training_set = create_octahedron_training_set(30, 6, 1, stoichiometry)

In [21]:
ordering_energy_calculator = EC.BayesianRRCalculator('TFC')
ordering_energy_calculator.fit(training_set, 'EMT')
top_coef = ordering_energy_calculator.get_coefficients()

coefficients, _ = EC.compute_coefficients_for_linear_topological_model(top_coef, training_set[1].get_all_symbols(), training_set[1].get_n_atoms()) 
ordering_energy_calculator.set_coefficients(coefficients)
ordering_energy_calculator.set_feature_key('TEC')
ordering_energy_calculator.energy_key = 'ORD'

Coef symbol_a: Au


In [22]:
adsorbate_calculator = EC.BayesianRRCalculator('ADS')
coeff_ads = [0, 0,
             -0.28120172, -1.01410455, -2.23349599,
             -0.35726243, -1.00039431, -2.08995281, -2.40469383]
adsorbate_calculator.set_coefficients(coeff_ads)

In [23]:
local_environment_calculator = LocalEnvironmentCalculator()
local_feature_classifier = TopologicalEnvironmentClassifier(
                                                local_environment_calculator,
                                                ['Au', 'Pt'])

In [24]:
start_particle = create_start_particle(6, 1, {'Au' : 84, 'Pt' : 56})

In [25]:
kwargs = {
    'beta' : 200,
    'max_steps' : 20000,
    'start_particle' : start_particle,
    'ordering_energy_calculator' : ordering_energy_calculator, 
    'adsorbates_energy_calculator' : adsorbate_calculator,
    'n_adsorbates' : 4, 
    'local_feature_classifier' : local_feature_classifier}

[best_particle, accepted_energies, best_particles] = MonteCarlo.run_monte_carlo_ordering_adsorbates(**kwargs)

/home/riccardo/Projects/NanoParticleLibrary/Core/EnergyCalculator.py:282: RuntimeWarning: divide by zero encountered in true_divide
  interaction_matrix = self.a / distance_matrix**2
/home/riccardo/Projects/NanoParticleLibrary/MCMC/MonteCarlo.py:275: RuntimeWarning: overflow encountered in exp
  acceptance_rate = min(1, np.exp(-beta * delta_e))


Step: 2000
Lowest energy: -9.13135192798472
Step: 4000
Lowest energy: -9.132905323169872
Step: 6000
Lowest energy: -9.132905323169872
Step: 8000
Lowest energy: -9.172339007133184
Step: 10000
Lowest energy: -9.172339007133184
Step: 12000
Lowest energy: -9.174075638979959
Step: 14000
Lowest energy: -9.174075638979959
Step: 16000
Lowest energy: -9.174075638979959
Step: 18000
Lowest energy: -9.1759384338565
Step: 20000
Lowest energy: -9.1759384338565
Step: 22000
Lowest energy: -9.1759384338565
Step: 24000
Lowest energy: -9.17971938337014
Step: 26000
Lowest energy: -9.17971938337014
Step: 28000
Lowest energy: -9.17971938337014
Step: 30000
Lowest energy: -9.17971938337014
Step: 32000
Lowest energy: -9.17971938337014
Step: 34000
Lowest energy: -9.17971938337014
Step: 36000
Lowest energy: -9.17971938337014
Step: 38000
Lowest energy: -9.179779858116099
Step: 40000
Lowest energy: -9.179779858116099
Step: 42000
Lowest energy: -9.179779858116099
Step: 44000
Lowest energy: -9.179779858116099
Step: 

In [19]:
from Core import Adsorption
from ase.visualize import view

ads_placer = Adsorption.PlaceAddAtoms(start_particle.get_all_symbols())
ads_placer.bind_particle(start_particle)
ads_site_list = start_particle.get_adsorption_list()

ads_positions = [ads_site_list[site] for site in best_particle['ads']] 
ads_placer.place_add_atom(start_particle, 'O', ads_positions)
view(start_particle.get_ase_atoms(), viewer='x3d')

In [20]:
for ads_num in range(2,11):
    start_particle = create_start_particle(6, 1, {'Ni' : 84, 'Au' : 56})
    kwargs['n_adsorbates'] = ads_num
    [best_particle, accepted_energies, best_particles] = MonteCarlo.run_monte_carlo_ordering_adsorbates(**kwargs)
    best_p = get_np_with_ads_from_dict(best_particle)
    best_p.write('/home/riccardo/Stuff_toPass/NP_2_ads_{}.xyz'.format(ads_num))

Step: 2000
Lowest energy: 18.849751017360653
Step: 4000
Lowest energy: 18.347579826627477
Step: 6000
Lowest energy: 18.268057605621447
Step: 2000
Lowest energy: 16.255949481861272
Step: 4000
Lowest energy: 15.83421568241646
Step: 6000
Lowest energy: 15.816769216219909
Step: 2000
Lowest energy: 13.900643806166906
Step: 4000
Lowest energy: 13.597067685396754
Step: 2000
Lowest energy: 11.328590710495835
Step: 4000
Lowest energy: 11.29683342674085
Step: 2000
Lowest energy: 9.401540923515867
Step: 4000
Lowest energy: 9.252180850715725
Step: 6000
Lowest energy: 9.229051741407444
Step: 2000
Lowest energy: 8.373006754592943
Step: 4000
Lowest energy: 8.077595982331786
Step: 6000
Lowest energy: 7.466346024129997
Step: 8000
Lowest energy: 7.325725828670265
Step: 2000
Lowest energy: 7.499228147687814
Step: 4000
Lowest energy: 6.620522310570834
Step: 6000
Lowest energy: 5.789668324188066
Step: 8000
Lowest energy: 5.5611864857021995
Step: 10000
Lowest energy: 5.412616425747094
Step: 2000
Lowest ener

In [51]:
import copy 

bare_np = get_np_with_ads_from_dict(best_particle)
ads_list = bare_np.get_adsorption_list()

hollow_position = [list(ads_list[x]) for x in range(bare_np.get_total_number_of_sites()) if len(ads_list[x]) == 3]
hollow_position_ni = [x for x in hollow_position if ''.join(p.get_symbols(x)) == 'NiNiNi']
ads_placer = Adsorption.PlaceAddAtoms(bare_np.get_all_symbols())
ads_placer.bind_particle(bare_np)
count=1
for f in hollow_position_ni[1:]:    
    p = copy.deepcopy(bare_np)
    p = ads_placer.place_add_atom(p, 'O', [hollow_position_ni[0], f])
    atoms = p.get_ase_atoms()
    dist = round(atoms.get_distance(140,141),3)
    p.write('/home/riccardo/Stuff_toPass/NP.{}_{}.xyz'.format(count, dist))
    count +=1


In [50]:
len(hollow_position_ni)

24

In [16]:
from Core import Adsorption
from ase.visualize import view

def get_np_with_ads_from_dict(best_particle):
    best_p = NP.Nanoparticle()
    best_p.build_from_dictionary(best_particle)
    best_p.construct_neighbor_list()
    best_p.construct_adsorption_list()

    ads_placer = Adsorption.PlaceAddAtoms(best_p.get_all_symbols())
    ads_placer.bind_particle(best_p)
    ads_site_list = best_p.get_adsorption_list()

    ads_positions = [list(ads_site_list[site]) for site in best_particle['ads']] 
    ads_placer.place_add_atom(best_p, 'O', ads_positions)
    return best_p